1) m = 0, $s^2$ = 9; $\chi^2$
-распределение с m степенями свободы ($\chi^2_m$), m = 4. Стьюдента с m
степенями свободы (${t}_m$), m = 6.

In [49]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
%matplotlib inline

def MCM(alpha_t, beta, M):
    while True:
        alpha_t = (alpha_t * beta) % M
        yield alpha_t / M

mcm_gen = MCM(131075, 131075, 2**31)

M = 0
S = 3
df = 4
m = 6
N = 1000

Normal distribution

In [50]:
def normal(gen, m, std):
    while True:
        yield stats.norm.ppf(next(gen), m, std)

normal_gen = normal(mcm_gen, M, S)
normal_vals = [next(normal_gen) for _ in range(N)]

sample_mean = np.mean(normal_vals)
sample_var = np.var(normal_vals) * N / (N - 1)
print('Sample mean:', sample_mean)
print('Sample variance:', sample_var)

print('Expected mean:', M)
print('Expected variance:', S**2)

Sample mean: -0.0321984701017165
Sample variance: 9.210191477299528
Expected mean: 0
Expected variance: 9


Chi square distribution

In [51]:
def chi_square(gen, df):
    while True:
        yield stats.chi2.ppf(next(gen),df)

chi2_gen = chi_square(mcm_gen, df)
chi2_vals = [next(chi2_gen) for _ in range(N)]

sample_mean = np.mean(chi_vals)
sample_var = np.var(chi_vals) * N / (N - 1)
print('Sample mean:', sample_mean)
print('Sample variance:', sample_var)

print('Expected mean:', df)
print('Expected variance:', df*2)

Sample mean: 4.046113590686566
Sample variance: 8.248602943071452
Expected mean: 4
Expected variance: 8


Sudent distribution

In [52]:
import math

def student(gen, m):
    while True:
        yield stats.t.ppf(next(gen),m)

student_gen = student(mcm_gen, m)
student_vals = [next(student_gen) for _ in range(N)]

sample_mean = np.mean(student_vals)
sample_var = np.var(student_vals) * N / (N - 1)
print('Sample mean:', sample_mean)
print('Sample variance:', sample_var)


print('Expected mean:', 0)
print('Expected variance:', m/(m-2))

Sample mean: 0.019964858814899195
Sample variance: 1.4215956195500268
Expected mean: 0
Expected variance: 1.5


In [53]:
def kolmogorov_criterion(gen, cdf = stats.uniform.cdf, number_of_samples = N):
    return stats.kstest([next(gen) for _ in range(number_of_samples)], cdf)

In [54]:
from tqdm import trange

normal_cdf = lambda x: stats.norm.cdf(x, M, S)

print(f'Kolmogorov criterion for normal distribution with mean = {M} and std = {S}:')
print(kolmogorov_criterion(normal_gen, normal_cdf))

print('\nChecking if significance level tends to 0.05:')
NUMBER_OF_TESTS = 1000
number_of_errors = 0
for i in trange(NUMBER_OF_TESTS):
    if kolmogorov_criterion(normal_gen, normal_cdf).pvalue < 0.05:
        number_of_errors += 1
print('Approximate significance level:', number_of_errors / NUMBER_OF_TESTS)

Kolmogorov criterion for normal distribution with mean = 0 and std = 3:
KstestResult(statistic=0.03231728673353795, pvalue=0.24195363027077132, statistic_location=2.154597102170484, statistic_sign=1)

Checking if significance level tends to 0.05:


100%|██████████| 1000/1000 [01:39<00:00, 10.08it/s]

Approximate significance level: 0.054


In [55]:
chi2_cdf = lambda x: stats.chi2.cdf(x, df)

print(f'Kolmogorov criterion for chi2 distribution with degrees of freedom = {df}:')
print(kolmogorov_criterion(chi2_gen, chi2_cdf))

print('\nChecking if significance level tends to 0.05:')
number_of_errors = 0
for i in trange(NUMBER_OF_TESTS):
    if kolmogorov_criterion(chi2_gen, chi2_cdf).pvalue < 0.05:
        number_of_errors += 1
print('Approximate significance level:', number_of_errors / NUMBER_OF_TESTS)

Kolmogorov criterion for chi2 distribution with degrees of freedom = 4:
KstestResult(statistic=0.03672031651809804, pvalue=0.1315201906983986, statistic_location=4.218205294574568, statistic_sign=-1)

Checking if significance level tends to 0.05:


100%|██████████| 1000/1000 [01:50<00:00,  9.04it/s]

Approximate significance level: 0.041


In [56]:
student_cdf = lambda x: stats.t.cdf(x, m)

print(f'Kolmogorov criterion for student distribution with m = {m}')
print(kolmogorov_criterion(student_gen, student_cdf))

print('\nChecking if significance level tends to 0.05:')
number_of_errors = 0
for i in trange(NUMBER_OF_TESTS):
    if kolmogorov_criterion(student_gen, student_cdf).pvalue < 0.05:
        number_of_errors += 1
print('Approximate significance level:', number_of_errors / NUMBER_OF_TESTS)

Kolmogorov criterion for student distribution with m = 6
KstestResult(statistic=0.02741459174082972, pvalue=0.43231061959314165, statistic_location=0.28554339534752177, statistic_sign=1)

Checking if significance level tends to 0.05:


100%|██████████| 1000/1000 [02:01<00:00,  8.24it/s]

Approximate significance level: 0.047


In [57]:
import math

NUMBER_OF_BINS = 100

def chisquare_criterion(gen, min_val, max_val, cdf):
    bin_counts = [0] * (NUMBER_OF_BINS + 2)
    segment = (max_val - min_val) / NUMBER_OF_BINS

    for i in range(N):
        gen_number = next(gen)
        if gen_number < min_val:
            bin_counts[0] += 1
        elif gen_number > max_val:
            bin_counts[-1] += 1
        else:
            bin_ind = math.floor((gen_number - min_val) / segment)
            bin_counts[bin_ind + 1] += 1

    exp_probs = [0] * (NUMBER_OF_BINS + 2)
    exp_probs[0] = cdf(min_val)
    exp_probs[-1] = 1. - cdf(max_val)

    for i in range(NUMBER_OF_BINS):
        exp_probs[i + 1] = cdf(min_val + (i + 1) * segment) - cdf(min_val + i * segment)

    for i in range(len(exp_probs)):
        exp_probs[i] *= N

    return stats.chisquare(bin_counts, exp_probs)

In [58]:
min_val = M - S
max_val = M + S

print(f'Chisquare criterion for normal distribution with mean = {M} and std = {S}:')
print(chisquare_criterion(normal_gen, min_val, max_val, normal_cdf))

print('\nChecking if significance level tends to 0.05:')
number_of_errors = 0
for i in trange(NUMBER_OF_TESTS):
    if chisquare_criterion(normal_gen, min_val, max_val, normal_cdf).pvalue < 0.05:
        number_of_errors += 1
print('Approximate significance level:', number_of_errors / NUMBER_OF_TESTS)

Chisquare criterion for normal distribution with mean = 0 and std = 3:
Power_divergenceResult(statistic=96.73476760094458, pvalue=0.6015406012773878)

Checking if significance level tends to 0.05:


100%|██████████| 1000/1000 [02:13<00:00,  7.48it/s]

Approximate significance level: 0.062


In [61]:
min_val = 1
max_val = 8

print(f'Chisquare criterion for chi2 distribution with df = {df}:')
print(chisquare_criterion(chi2_gen, min_val, max_val, chi2_cdf))

print('\nChecking if significance level tends to 0.05:')
number_of_errors = 0
for i in trange(NUMBER_OF_TESTS):
    if chisquare_criterion(chi2_gen, min_val, max_val, chi2_cdf).pvalue < 0.05:
        number_of_errors += 1
print('Approximate significance level:', number_of_errors / NUMBER_OF_TESTS)

Chisquare criterion for chi2 distribution with df = 4:
Power_divergenceResult(statistic=64.91976534555602, pvalue=0.9979926013755582)

Checking if significance level tends to 0.05:


100%|██████████| 1000/1000 [02:28<00:00,  6.73it/s]

Approximate significance level: 0.053


In [60]:
min_val = -5
max_val = 5

print(f'Chisquare criterion for student distribution with m = {m}:')
print(chisquare_criterion(student_gen, min_val, max_val, student_cdf))

print('\nChecking if significance level tends to 0.05:')
number_of_errors = 0
for i in trange(NUMBER_OF_TESTS):
    if chisquare_criterion(student_gen, min_val, max_val, student_cdf).pvalue < 0.05:
        number_of_errors += 1
print('Approximate significance level:', number_of_errors / NUMBER_OF_TESTS)

Chisquare criterion for student distribution with m = 6:
Power_divergenceResult(statistic=89.49080748614433, pvalue=0.7868023826521545)

Checking if significance level tends to 0.05:


100%|██████████| 1000/1000 [02:19<00:00,  7.19it/s]

Approximate significance level: 0.093
